In [1]:
import os
import datetime
from google.oauth2.service_account import Credentials
import pandas_gbq
import math
import time
import re

In [3]:
# insert data to mysql
import MySQLdb
import pandas as pd
from sqlalchemy import create_engine
import pymysql

user='XXXXX'
password='XXXXX'
host='XXX.XXX.XXX.XXX'
port=3306
db='XXXXX'
           
def insert_to_mysql(df, table_name):
    engine =create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + str(port) + '/' + db , echo=False)
#     engine = create_engine(str(r"mysql+mysqldb://%s:" + '%s' + "@%s/%s") % (user, password, host, db))

    try:
        df.to_sql(table_name,con=engine,if_exists='replace',index=False) #if_exists='append'
        print("Successfully insert.")
    except Exception as e:
        print(e)

In [4]:
def query_table(date_name, project_id, query_name):
    #import code
    creds = Credentials.from_service_account_file('/Users/USER/Desktop/GA_firebase/code/warehouse/GA_Api_for_Intern_New.json')
    os.chdir("/Users/USER/Desktop/GA_firebase/query/Warehouse")
    query_table = ''
    with open(query_name + ".txt", "r", encoding="utf-8") as file:
        query_table = file.read().format(date = date_name)
        file.close()
    #pull data
    df_daily ={}
    df_daily = pandas_gbq.read_gbq(query_table, project_id = project_id, credentials = creds, dialect="standard")
    return df_daily
query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste")

Downloading: 100%|█████████████████████████████████████████████████████████████| 9859/9859 [00:02<00:00, 4133.43rows/s]


,event_date,event_operatingsystem,event_device,event_name,event_action,event_label,TotalEvents
0,20200712,Android,com.tvbs.news.food.topnews.bestfood.restaurant...,click_collect,article_collect,1秒出國！苗栗全新度假酒店坐擁無敵景觀，日式榻榻米房還有超大獨立浴池_325326_收藏_文...,3
1,20200712,Android,com.tvbs.news.food.topnews.bestfood.restaurant...,click_collect,article_collect,5/27節目《2天1夜go》店家資訊：小琉球升級版玩法+網路限定版影音_324674_收藏_...,1
2,20200712,Android,com.tvbs.news.food.topnews.bestfood.restaurant...,click_collect,article_collect,6/23節目《食尚玩家》店家資訊：主廚美食口袋清單_325089_收藏_文章內頁_節目,1
3,20200712,Android,com.tvbs.news.food.topnews.bestfood.restaurant...,click_collect,article_collect,6/8節目《熱血48小時》店家資訊：全台排隊幽靈美食_324840_收藏_文章內頁_節目,1
4,20200712,Android,com.tvbs.news.food.topnews.bestfood.restaurant...,click_collect,article_collect,7/1起夜騎深澳鐵道囉！搭「河豚車」賞水母燈海、星空隧道，搭三倍劵再省100元_325154...,1
...,...,...,...,...,...,...,...
9854,20200712,iOS,tw.com.tvbs.news.food.topnews.bestfood.restaur...,show_screen,Store Detail Page,龍泉冰店_13563_店家資訊頁,1
9855,20200712,iOS,tw.com.tvbs.news.food.topnews.bestfood.restaur...,show_screen,Store Detail Page,龍洞灣岬步道_13388_店家資訊頁,1
9856,20200712,iOS,tw.com.tvbs.news.food.topnews.bestfood.restaur...,show_screen,Store Detail Page,龍祥鴨莊\t_10490_店家資訊頁,1
9857,20200712,iOS,tw.com.tvbs.news.food.topnews.bestfood.restaur...,show_screen,Store Detail Page,龍門海鮮_12905_店家資訊頁,1


In [5]:
# PV for ios and android
def query_table(date_name, project_id, query_name):
    #import code
    creds = Credentials.from_service_account_file('/Users/USER/Desktop/GA_firebase/code/warehouse/GA_Api_for_Intern_New.json')
    query_table = ''
    with open("/Users/USER/Desktop/GA_firebase/query/Warehouse/ios_android_query.txt", "r", encoding="utf-8") as file:
        query_table = file.read().format(date = date_name)
        file.close()
    #pull data
    df_daily ={}
    df_daily = pandas_gbq.read_gbq(query_table, project_id = project_id, credentials = creds, dialect="standard")
    return df_daily

query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste")
df_PV = pd.DataFrame(query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste"), columns= ['event_date', 'event_operatingsystem', 'event_num'])
df_PV 

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.88rows/s]


,event_date,event_operatingsystem,event_num
0,20200712,ANDROID,66325
1,20200712,IOS,78792


In [6]:
insert_to_mysql(df_PV,'ios_android_PV')

Successfully insert.


In [7]:
# detail push
def query_table(date_name, project_id, query_name):
    #import code
    creds = Credentials.from_service_account_file('/Users/USER/Desktop/GA_firebase/code/warehouse/GA_Api_for_Intern_New.json')
    query_table = ''
    with open("/Users/USER/Desktop/GA_firebase/query/Warehouse/detail_push_query.txt", "r", encoding="utf-8") as file:
        query_table = file.read().format(date = date_name)
        file.close()
    #pull data
    df_daily ={}
    df_daily = pandas_gbq.read_gbq(query_table, project_id = project_id, credentials = creds, dialect="standard")
    return df_daily
query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste")
df_detail_push = pd.DataFrame(query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste"), columns= ['event_date','event_name','event_num','title','article_id','article','article_category'])
df_detail_push

Downloading: 100%|███████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 93.29rows/s]


,event_date,event_name,event_num,title,article_id,article,article_category
0,20200712,detail_push,1581,坐擁無敵海景的峇里島風民宿馬上抽，100張住宿券每週抽！,9,搖一搖,推播
1,20200712,detail_push,1470,手腳慢也能搶！4張信用卡「振興三倍券」回饋好康，全都「沒有名額限制」,325297,文章,推播
2,20200712,detail_push,1377,"比單搭高鐵還便宜！「高鐵假期」來回車票+住宿每人2,450元起，加碼「暑期親子專案」遛小孩免動腦",325261,文章,推播
3,20200712,detail_push,1069,夜貓族開吃！麥當勞推「深夜10塊雞塊餐」，16塊麥克雞塊、大薯、飲料現省63元,325282,文章,推播
4,20200712,detail_push,1019,衝IKEA吃爆！一支20元「芒果霜淇淋」夏季回歸，還有5款水果冰棒一併買,325302,文章,推播
5,20200712,detail_push,261,7/6節目《熱血48小時》店家資訊：夏天澎湖新玩法,325252,文章,推播
6,20200712,detail_push,100,有「7」就打折，210元吃7道菜鐵板燒，這3種號碼都可對！高雄新店加碼「買一送一」,325286,文章,推播
7,20200712,detail_push,52,馬上抽用心打造別具一格的旅店：嘉楠風華酒店，共100張住宿券每週抽！,9,搖一搖,推播
8,20200712,detail_push,24,【新開店】全台首家「海洋元素飯店」！試營運吃到飽1人免費、房內可看「水族館」、郵輪酒吧買一送一,325307,文章,推播
9,20200712,detail_push,15,果粉嚇歪iPhone 12「變很小」！走回第一代「單手操控」尺寸，比女生手掌還小,325317,文章,推播


In [8]:
insert_to_mysql(df_detail_push, 'detail_push')

Successfully insert.


In [9]:
# show screen
def query_table(date_name, project_id, query_name):
    #import code
    creds = Credentials.from_service_account_file('/Users/USER/Desktop/GA_firebase/code/warehouse/GA_Api_for_Intern_New.json')
    query_table = ''
    with open("/Users/USER/Desktop/GA_firebase/query/Warehouse/show_screen_query.txt", "r", encoding="utf-8") as file:
        query_table = file.read().format(date = date_name)
        file.close()
    #pull data
    df_daily ={}
    df_daily = pandas_gbq.read_gbq(query_table, project_id = project_id, credentials = creds, dialect="standard")
    return df_daily
query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste")
df_show_screen = pd.DataFrame(query_table(20200712, 'supertaste-tvbs-fc57d', "bigquery_supertaste"), columns= ['event_date','event_name','event_num','title','article_id','article','article_category'])
df_show_screen

Downloading: 100%|█████████████████████████████████████████████████████████████| 1938/1938 [00:01<00:00, 1031.58rows/s]


,event_date,event_name,event_num,title,article_id,article,article_category
0,20200712,show_screen,6993,食尚APP送你100張住宿券！這週抽6家：台北溫泉時尚房、恆春療癒海景房,325268,文章內頁,懶人包
1,20200712,show_screen,2565,7/9節目《瘋狂總部》店家資訊：72小時瘋遊高屏,325320,文章內頁,節目
2,20200712,show_screen,1603,手腳慢也能搶！4張信用卡「振興三倍券」回饋好康，全都「沒有名額限制」,325297,文章內頁,三倍券
3,20200712,show_screen,1558,"比單搭高鐵還便宜！「高鐵假期」來回車票+住宿每人2,450元起，加碼「暑期親子專案」遛小孩免動腦",325261,文章內頁,旅行
4,20200712,show_screen,1158,夜貓族開吃！麥當勞推「深夜10塊雞塊餐」，16塊麥克雞塊、大薯、飲料現省63元,325282,文章內頁,美食
...,...,...,...,...,...,...,...
1933,20200712,show_screen,1,天團特搜 彰化美食奇聞追追追,323756,文章內頁,節目
1934,20200712,show_screen,1,一路吃到底！彰化美食傳說PART2,322714,文章內頁,節目
1935,20200712,show_screen,1,地毯式搜索 彰化美食又來了,323201,文章內頁,節目
1936,20200712,show_screen,1,台版《魔戒》場景！美拍5大夢幻「哈比屋」小村，還能逛粉紅城堡、彩虹階梯,321401,文章內頁,懶人包


In [10]:
insert_to_mysql(df_show_screen, 'show_screen_data')

Successfully insert.
